# Distance from Channel Capacity: Interactive Exploration

This notebook provides interactive tools to explore the channel capacity distance framework for quantum gravity.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, HTML
import scipy.linalg as la
%matplotlib inline

## 1. Channel Capacity Basics

Explore how different quantum channels affect the capacity and resulting distance metric.

In [ ]:
def depolarizing_capacity(p):
    """Capacity of depolarizing channel as function of noise parameter p"""
    if p >= 2/3:
        return 0
    else:
        # Binary entropy function
        def h2(x):
            if x == 0 or x == 1:
                return 0
            return -x*np.log2(x) - (1-x)*np.log2(1-x)
        
        return 1 - h2((1+p)/2)

def amplitude_damping_capacity(gamma):
    """Capacity of amplitude damping channel"""
    if gamma == 0:
        return 1
    elif gamma == 1:
        return 0
    else:
        # Simplified approximation
        return (1 - gamma) * np.log2(2/(1 + gamma))

# Interactive plot
@widgets.interact(channel_type=['Depolarizing', 'Amplitude Damping'])
def plot_capacity_distance(channel_type='Depolarizing'):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    if channel_type == 'Depolarizing':
        p_values = np.linspace(0, 1, 100)
        capacities = [depolarizing_capacity(p) for p in p_values]
        distances = [-np.log(c) if c > 0 else 10 for c in capacities]
        
        ax1.plot(p_values, capacities, 'b-', linewidth=2)
        ax1.set_xlabel('Noise parameter p')
        ax1.set_ylabel('Channel Capacity C(p)')
        ax1.set_title('Depolarizing Channel Capacity')
        ax1.grid(True, alpha=0.3)
        ax1.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        ax1.axvline(x=2/3, color='r', linestyle='--', label='Classical threshold')
        ax1.legend()
        
        ax2.plot(p_values, distances, 'r-', linewidth=2)
        ax2.set_xlabel('Noise parameter p')
        ax2.set_ylabel('Distance d = -log C(p)')
        ax2.set_title('Information Distance')
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim([0, 10])
        
    else:  # Amplitude Damping
        gamma_values = np.linspace(0, 1, 100)
        capacities = [amplitude_damping_capacity(g) for g in gamma_values]
        distances = [-np.log(c) if c > 0 else 10 for c in capacities]
        
        ax1.plot(gamma_values, capacities, 'g-', linewidth=2)
        ax1.set_xlabel('Damping parameter γ')
        ax1.set_ylabel('Channel Capacity C(γ)')
        ax1.set_title('Amplitude Damping Channel Capacity')
        ax1.grid(True, alpha=0.3)
        ax1.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        
        ax2.plot(gamma_values, distances, 'r-', linewidth=2)
        ax2.set_xlabel('Damping parameter γ')
        ax2.set_ylabel('Distance d = -log C(γ)')
        ax2.set_title('Information Distance')
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim([0, 10])
    
    plt.tight_layout()
    plt.show()

## 2. Emergent Causal Structure

Visualize how causal structure emerges from channel capacity networks.

In [ ]:
@widgets.interact(n_events=(3, 10, 1), 
                 connection_prob=(0.1, 1.0, 0.1),
                 show_distances=True)
def visualize_causal_network(n_events=5, connection_prob=0.5, show_distances=True):
    """Interactive visualization of causal network structure"""
    
    # Generate random events in 2D spacetime
    np.random.seed(42)  # For reproducibility
    times = np.sort(np.random.uniform(0, 10, n_events))
    positions = np.random.uniform(-5, 5, n_events)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Draw causal connections
    for i in range(n_events):
        for j in range(i+1, n_events):
            dt = times[j] - times[i]
            dx = positions[j] - positions[i]
            
            # Check if causally connected (within light cone)
            if dt > 0 and abs(dx) <= dt:  # c = 1
                if np.random.random() < connection_prob:
                    # Calculate channel capacity based on interval
                    interval = dt**2 - dx**2
                    capacity = np.exp(-interval/10) * (1 - 0.1*np.random.random())
                    
                    # Draw connection with thickness proportional to capacity
                    ax.plot([positions[i], positions[j]], 
                           [times[i], times[j]], 
                           'b-', alpha=capacity, 
                           linewidth=3*capacity)
                    
                    if show_distances:
                        distance = -np.log(capacity)
                        mid_x = (positions[i] + positions[j])/2
                        mid_t = (times[i] + times[j])/2
                        ax.text(mid_x, mid_t, f'{distance:.2f}', 
                               fontsize=8, alpha=0.7)
    
    # Draw events
    ax.scatter(positions, times, s=100, c='red', zorder=5, edgecolors='black')
    
    # Label events
    for i in range(n_events):
        ax.annotate(f'E{i}', (positions[i], times[i]), 
                   xytext=(5, 5), textcoords='offset points')
    
    # Draw light cone from origin
    t_cone = np.linspace(0, 10, 100)
    ax.plot(t_cone, t_cone, 'g--', alpha=0.3, label='Light cone')
    ax.plot(-t_cone, t_cone, 'g--', alpha=0.3)
    
    ax.set_xlabel('Space x', fontsize=12)
    ax.set_ylabel('Time t', fontsize=12)
    ax.set_title('Emergent Causal Structure from Channel Capacity', fontsize=14)
    ax.grid(True, alpha=0.2)
    ax.legend()
    ax.set_xlim([-6, 6])
    ax.set_ylim([-0.5, 10.5])
    
    plt.show()

## 3. Black Hole Information Geometry

Explore how channel capacity behaves near a black hole horizon.

In [ ]:
@widgets.interact(r_horizon=(0.5, 3.0, 0.5),
                 show_component=['Both', 'Capacity', 'Distance'])
def black_hole_capacity_profile(r_horizon=2.0, show_component='Both'):
    """Visualize channel capacity near black hole"""
    
    r = np.linspace(0.1, 5*r_horizon, 200)
    
    # Schwarzschild metric component
    f = np.ones_like(r)
    outside = r > r_horizon
    f[outside] = np.sqrt(1 - r_horizon/r[outside])
    f[~outside] = 0.01  # Small but nonzero inside
    
    # Channel capacities (asymmetric)
    capacity_out = f**2  # Outward capacity suppressed by redshift
    capacity_in = f * (1 + 0.5*(1-f))  # Inward capacity less suppressed
    
    # At horizon, outward capacity vanishes
    horizon_idx = np.argmin(np.abs(r - r_horizon))
    capacity_out[r <= r_horizon] = 0.001
    
    # Information distances
    distance_out = -np.log(np.maximum(capacity_out, 1e-10))
    distance_in = -np.log(np.maximum(capacity_in, 1e-10))
    
    if show_component == 'Both':
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    else:
        fig, ax1 = plt.subplots(figsize=(10, 6))
        if show_component == 'Distance':
            ax1 = ax2 = ax1
    
    if show_component in ['Both', 'Capacity']:
        ax1.plot(r, capacity_out, 'b-', linewidth=2, label='Outward C(r→∞)')
        ax1.plot(r, capacity_in, 'r--', linewidth=2, label='Inward C(∞→r)')
        ax1.axvline(x=r_horizon, color='k', linestyle=':', linewidth=2, label=f'Horizon (r={r_horizon})')
        ax1.fill_between(r[r <= r_horizon], 0, 1, alpha=0.2, color='gray', label='Inside horizon')
        ax1.set_xlabel('Radius r', fontsize=12)
        ax1.set_ylabel('Channel Capacity', fontsize=12)
        ax1.set_title('Channel Capacity Near Black Hole', fontsize=14)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.set_ylim([0, 1.2])
    
    if show_component in ['Both', 'Distance']:
        ax2.plot(r, distance_out, 'b-', linewidth=2, label='Outward d(r→∞)')
        ax2.plot(r, distance_in, 'r--', linewidth=2, label='Inward d(∞→r)')
        ax2.axvline(x=r_horizon, color='k', linestyle=':', linewidth=2, label=f'Horizon (r={r_horizon})')
        ax2.fill_between(r[r <= r_horizon], 0, 20, alpha=0.2, color='gray', label='Inside horizon')
        ax2.set_xlabel('Radius r', fontsize=12)
        ax2.set_ylabel('Information Distance', fontsize=12)
        ax2.set_title('Information-Theoretic Distance', fontsize=14)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim([0, 20])
    
    plt.tight_layout()
    plt.show()
    
    print(f"Key observations:")
    print(f"• Outward capacity vanishes at horizon → no information escape")
    print(f"• Inward capacity remains finite → information can fall in")
    print(f"• Asymmetry naturally explains black hole information paradox")

## 4. Holographic Entropy from Channel Counting

Demonstrate how the Bekenstein-Hawking entropy formula emerges from boundary channel counting.

In [ ]:
@widgets.interact(radius=(1, 20, 1),
                 planck_length=(0.5, 2.0, 0.1))
def holographic_entropy_calculation(radius=10, planck_length=1.0):
    """Calculate entropy from boundary channel counting"""
    
    # Surface area of sphere
    area = 4 * np.pi * radius**2
    
    # Number of channels (one per Planck area)
    planck_area = planck_length**2
    n_channels = int(area / (4 * planck_area))  # Factor of 4 for entropy bound
    
    # Each channel carries ~1 bit
    bits_per_channel = 1.0
    total_information = n_channels * bits_per_channel
    
    # Bekenstein-Hawking entropy
    bh_entropy = area / (4 * planck_area)
    
    # Visualization
    fig = plt.figure(figsize=(14, 6))
    
    # 3D sphere with channels
    ax1 = fig.add_subplot(121, projection='3d')
    
    # Draw sphere
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x = radius * np.outer(np.cos(u), np.sin(v))
    y = radius * np.outer(np.sin(u), np.sin(v))
    z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
    
    ax1.plot_surface(x, y, z, alpha=0.3, color='blue')
    
    # Sample some channel points on surface
    n_display = min(100, n_channels)
    theta = np.random.uniform(0, 2*np.pi, n_display)
    phi = np.arccos(1 - 2*np.random.uniform(0, 1, n_display))
    
    cx = radius * np.sin(phi) * np.cos(theta)
    cy = radius * np.sin(phi) * np.sin(theta)
    cz = radius * np.cos(phi)
    
    ax1.scatter(cx, cy, cz, c='red', s=20, alpha=0.6)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(f'Boundary Channels\n(showing {n_display} of {n_channels} channels)')
    
    # Entropy comparison
    ax2 = fig.add_subplot(122)
    
    categories = ['Channel\nCounting', 'Bekenstein-\nHawking']
    values = [total_information, bh_entropy]
    colors = ['blue', 'red']
    
    bars = ax2.bar(categories, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    
    # Add value labels on bars
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.1f} bits',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax2.set_ylabel('Entropy (bits)', fontsize=12)
    ax2.set_title('Holographic Entropy Comparison', fontsize=14)
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Results for sphere with radius r = {radius} ℓ_P:")
    print(f"• Surface area: A = {area:.1f} ℓ_P²")
    print(f"• Number of channels: N = {n_channels}")
    print(f"• Information from channels: I = {total_information:.1f} bits")
    print(f"• Bekenstein-Hawking: S = A/4 = {bh_entropy:.1f} bits")
    print(f"• Ratio: I/S = {total_information/bh_entropy:.3f}")
    print(f"\nThe agreement confirms holographic principle from channel counting!")

## 5. Metric Properties Analysis

Systematically verify the metric properties of the distance function.

In [ ]:
def verify_metric_properties_detailed(n_events=4, noise_level=0.1):
    """Detailed verification of metric properties"""
    
    print(f"Testing metric properties with {n_events} events...\n")
    
    # Create capacity matrix (asymmetric for Lorentzian signature)
    C = np.random.uniform(0.1, 1.0, (n_events, n_events))
    np.fill_diagonal(C, 1.0)  # Perfect self-channel
    
    # Add noise to break symmetry
    C = C + noise_level * np.random.randn(n_events, n_events)
    C = np.clip(C, 0.01, 1.0)
    
    # Calculate distance matrix
    D = -np.log(C)
    
    # 1. Non-negativity
    print("1. NON-NEGATIVITY: d(i,j) ≥ 0")
    non_neg = np.all(D >= 0)
    print(f"   Result: {non_neg}")
    print(f"   Min distance: {np.min(D):.4f}\n")
    
    # 2. Identity
    print("2. IDENTITY: d(i,i) = 0")
    identity = np.allclose(np.diag(D), 0, atol=1e-6)
    print(f"   Result: {identity}")
    print(f"   Max self-distance: {np.max(np.abs(np.diag(D))):.6f}\n")
    
    # 3. Symmetry (generally violated for Lorentzian)
    print("3. SYMMETRY: d(i,j) = d(j,i)")
    asymmetry = D - D.T
    max_asymmetry = np.max(np.abs(asymmetry))
    print(f"   Result: {np.allclose(D, D.T, atol=1e-6)}")
    print(f"   Max asymmetry: {max_asymmetry:.4f}")
    print(f"   → Lorentzian signature emerges from asymmetry!\n")
    
    # 4. Triangle inequality
    print("4. TRIANGLE INEQUALITY: d(i,k) ≤ d(i,j) + d(j,k)")
    violations = []
    for i in range(n_events):
        for j in range(n_events):
            for k in range(n_events):
                if i != j and j != k and i != k:
                    if D[i,k] > D[i,j] + D[j,k] + 1e-6:
                        violations.append((i,j,k, D[i,k] - D[i,j] - D[j,k]))
    
    if len(violations) == 0:
        print(f"   Result: SATISFIED")
    else:
        print(f"   Result: {len(violations)} violations found")
        print(f"   Largest violation: {max(v[3] for v in violations):.4f}")
        print(f"   Note: Modified inequality d(i,k) ≤ d(i,j) + d(j,k) + log(α) expected\n")
    
    # Visualize distance matrix
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Capacity matrix
    im1 = ax1.imshow(C, cmap='viridis', vmin=0, vmax=1)
    ax1.set_title('Channel Capacity Matrix C(i→j)', fontsize=12)
    ax1.set_xlabel('Target Event j')
    ax1.set_ylabel('Source Event i')
    plt.colorbar(im1, ax=ax1, label='Capacity')
    
    # Distance matrix
    im2 = ax2.imshow(D, cmap='hot', vmin=0, vmax=5)
    ax2.set_title('Distance Matrix d(i,j) = -log C(i→j)', fontsize=12)
    ax2.set_xlabel('Target Event j')
    ax2.set_ylabel('Source Event i')
    plt.colorbar(im2, ax=ax2, label='Distance')
    
    plt.tight_layout()
    plt.show()
    
    return D

# Run the verification
D_matrix = verify_metric_properties_detailed(n_events=5, noise_level=0.2)

## 6. Path to Publication

Summary of key results and next steps for publication.

### Key Novel Contributions

1. **Distance Metric**: $d(A,B) = -\alpha \log C(A \to B)$
   - First proposal using channel capacity for spacetime distance
   - Natural causal structure (infinite distance for spacelike)
   - Emergent Lorentzian signature from asymmetry

2. **Black Hole Physics**:
   - Channel capacity vanishes at horizon (no escape)
   - Asymmetric capacity explains information paradox
   - No need for firewalls or complementarity

3. **Einstein Equations**:
   - Derived from optimizing information flow
   - Information stress-energy tensor $T_{IC}^{\mu\nu}$
   - Unifies quantum information with gravity

4. **Holographic Principle**:
   - $S = A/4$ from boundary channel counting
   - Emerges without assuming AdS/CFT
   - Direct operational meaning

### Publication Strategy

**Phase 1: Letters Paper** (Current)
- Physical Review Letters submission
- Focus on distance metric and key results
- 4 pages + references

**Phase 2: Detailed Theory**
- Physical Review D full article
- Complete mathematical framework
- Quantum corrections and renormalization

**Phase 3: Applications**
- Cosmological implications
- Quantum communication experiments
- Connection to quantum error correction

### Open Questions

1. How to handle multipartite entanglement?
2. Connection to tensor networks and MERA?
3. Experimental tests in analog gravity systems?
4. Implications for quantum computing?

In [ ]:
# Generate summary figure for paper
fig = plt.figure(figsize=(15, 10))

# Create 2x2 subplot grid
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.25)

# Panel A: Channel capacity vs distance
ax1 = fig.add_subplot(gs[0, 0])
c_vals = np.linspace(0.01, 1, 100)
d_vals = -np.log(c_vals)
ax1.plot(c_vals, d_vals, 'b-', linewidth=2)
ax1.set_xlabel('Channel Capacity C', fontsize=11)
ax1.set_ylabel('Distance d = -log C', fontsize=11)
ax1.set_title('(a) Distance from Capacity', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.text(0.5, 3, r'$d(A,B) = -\alpha \log C(A \to B)$', fontsize=10, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.3))

# Panel B: Black hole profile
ax2 = fig.add_subplot(gs[0, 1])
r = np.linspace(0.5, 5, 100)
r_s = 2.0
f = np.sqrt(np.maximum(1 - r_s/r, 0))
ax2.plot(r, f**2, 'b-', linewidth=2, label='Outward')
ax2.plot(r, f*(1+0.3*(1-f)), 'r--', linewidth=2, label='Inward')
ax2.axvline(x=r_s, color='k', linestyle=':', linewidth=1.5)
ax2.set_xlabel('Radius r', fontsize=11)
ax2.set_ylabel('Channel Capacity', fontsize=11)
ax2.set_title('(b) Black Hole Horizon', fontsize=12, fontweight='bold')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

# Panel C: Causal network
ax3 = fig.add_subplot(gs[1, 0])
np.random.seed(42)
n = 7
x = np.random.randn(n)
y = np.random.randn(n)
ax3.scatter(x, y, s=100, c='red', zorder=5, edgecolors='black')
for i in range(n):
    for j in range(i+1, n):
        if np.random.random() < 0.3:
            ax3.plot([x[i], x[j]], [y[i], y[j]], 'b-', alpha=0.3, linewidth=1)
ax3.set_xlabel('Space', fontsize=11)
ax3.set_ylabel('Time', fontsize=11)
ax3.set_title('(c) Causal Network', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.set_aspect('equal')

# Panel D: Holographic entropy
ax4 = fig.add_subplot(gs[1, 1])
radii = np.linspace(1, 20, 20)
areas = 4 * np.pi * radii**2
entropy = areas / 4
ax4.plot(areas, entropy, 'g-', linewidth=2)
ax4.scatter(areas[::3], entropy[::3], c='green', s=50, zorder=5)
ax4.set_xlabel('Surface Area A', fontsize=11)
ax4.set_ylabel('Entropy S', fontsize=11)
ax4.set_title('(d) Holographic Bound', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.text(1000, 400, r'$S = \frac{A}{4\ell_P^2}$', fontsize=11,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.3))

plt.suptitle('Distance from Channel Capacity: Key Results', 
             fontsize=14, fontweight='bold', y=0.98)

plt.savefig('/home/claude/figure_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("Summary figure saved as 'figure_summary.png'")
print("Ready for inclusion in Physical Review Letters manuscript!")